In [1]:
import hashlib
from datetime import datetime

# Định nghĩa lớp Block
class Block:
    def __init__(self, message, previous_hash):
        self.message = message
        self.timestamp = str(datetime.now())
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update((self.message + self.timestamp + self.previous_hash).encode('utf-8'))
        return sha.hexdigest()

    def __repr__(self):
        return (
            f"Block(\n"
            f"    Message        : {self.message}\n"
            f"    Timestamp      : {self.timestamp}\n"
            f"    Previous Hash  : {self.previous_hash}\n"
            f"    Current Hash   : {self.hash}\n"
            f")\n"
        )

# Khởi tạo chuỗi blockchain với 3 khối
genesis_block = Block("This is the genesis block", "0")
blockchain = [genesis_block]

# Thêm block thứ 2: sẽ lấy hash của genesis block làm previous_hash
block2 = Block("This is block 2", genesis_block.hash)
blockchain.append(block2)

# Thêm block thứ 3: sẽ lấy hash của block 2 làm previous_hash
block3 = Block("This is block 3", block2.hash)
blockchain.append(block3)

# In ra trạng thái ban đầu của chuỗi
print("== Original Blockchain ==\n")
for i, blk in enumerate(blockchain):
    print(f"Block {i}:")
    print(blk)

# Giả lập việc “đánh tráo” nội dung của block thứ 2
print("== Tampering with Block 2's message ==\n")
blockchain[1].message = "This is a tampered block 2"
blockchain[1].hash = blockchain[1].calculate_hash()

# In ra hai khối sau khi tampering
print("== After Tampering ==\n")
print("Block 1 (Tampered):")
print(blockchain[1])
print("Block 2 (Depends on Block 1’s hash):")
print(blockchain[2])


== Original Blockchain ==

Block 0:
Block(
    Message        : This is the genesis block
    Timestamp      : 2025-05-31 23:12:51.737844
    Previous Hash  : 0
    Current Hash   : b49fd525542d36fd1b98c1e7bdd82369b983407be06c0751a4d4bd2a650d4a29
)

Block 1:
Block(
    Message        : This is block 2
    Timestamp      : 2025-05-31 23:12:51.737844
    Previous Hash  : b49fd525542d36fd1b98c1e7bdd82369b983407be06c0751a4d4bd2a650d4a29
    Current Hash   : 3cbc8ad1987c6cb146172a89bd2fc4afb4b127b429caee86a5c5db0cee8dedda
)

Block 2:
Block(
    Message        : This is block 3
    Timestamp      : 2025-05-31 23:12:51.737844
    Previous Hash  : 3cbc8ad1987c6cb146172a89bd2fc4afb4b127b429caee86a5c5db0cee8dedda
    Current Hash   : 2a1d4575780bde9bc200b0d7781aac4f6097c540edbfcadf1420bd05b9284542
)

== Tampering with Block 2's message ==

== After Tampering ==

Block 1 (Tampered):
Block(
    Message        : This is a tampered block 2
    Timestamp      : 2025-05-31 23:12:51.737844
    Previous

In [2]:
import hashlib

def sha256_hash(text: str) -> str:
    """
    Hàm nhận về chuỗi text, trả về mã băm SHA-256 dạng hex string.
    """
    h = hashlib.sha256()
    h.update(text.encode('utf-8'))
    return h.hexdigest()

def flip_first_char(s: str) -> str:
    """
    Thay đổi (đảo ngược) ký tự đầu tiên của chuỗi s.
    Cách làm: nếu ký tự đầu tiên khác '0' thì đổi thành '0', ngược lại đổi thành '1'.
    (Chỉ để minh hoạ “thay đổi một ký tự”, bạn có thể tuỳ chỉnh cách này tuỳ ý.)
    """
    if not s:
        return s
    first = s[0]
    new_first = '0' if first != '0' else '1'
    return new_first + s[1:]

if __name__ == "__main__":
    # Bước 1: Nhập một chuỗi bất kỳ
    original = input("Nhập một chuỗi bất kỳ: ")

    # Bước 2: Tính mã băm SHA-256 của chuỗi gốc
    hash_original = sha256_hash(original)
    print(f"\nChuỗi gốc         : {original}")
    print(f"Mã băm SHA-256    : {hash_original}")

    # Bước 3: Tạo chuỗi mới bằng cách thay đổi một ký tự (ở đây thay đổi ký tự đầu)
    tampered = flip_first_char(original)

    # Bước 4: Tính mã băm SHA-256 của chuỗi đã bị “thay đổi” một ký tự
    hash_tampered = sha256_hash(tampered)
    print(f"\nChuỗi bị thay đổi  : {tampered}")
    print(f"Mã băm SHA-256    : {hash_tampered}")

    # Bước 5: Nhận xét
    print("\n=> Chỉ cần thay đổi một ký tự nhỏ (ở đây là ký tự đầu tiên), mã băm SHA-256 đã thay đổi hoàn toàn so với trước.")



Chuỗi gốc         : Học phần Mạng máy tính
Mã băm SHA-256    : 65c684ecb5b1be63ed30fed125ee8ac1842cdc1b87f32a0772309f174608b2b2

Chuỗi bị thay đổi  : 0ọc phần Mạng máy tính
Mã băm SHA-256    : bc9097573c1f21da666c6db606f3d539c315d78e2fee96f6f86687b893d46386

=> Chỉ cần thay đổi một ký tự nhỏ (ở đây là ký tự đầu tiên), mã băm SHA-256 đã thay đổi hoàn toàn so với trước.


In [9]:
import json
from solcx import compile_standard, install_solc, set_solc_version
from web3 import Web3

# ================================
# 1. CÀI ĐẶT & CẤU HÌNH solc
# ================================
# Nếu chưa cài version 0.8.0, hãy bỏ comment dòng dưới để cài:
# install_solc("0.8.0")

# Chọn dùng solc 0.8.0 (phải khớp với pragma ^0.8.0 trong MessageStorage.sol)
set_solc_version("0.8.0")

# ================================
# 2. ĐỌC FILE MESSAGE STORAGE VÀ COMPILE
# ================================
with open("MessageStorage.sol", "r", encoding="utf-8") as f:
    source_contract = f.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "MessageStorage.sol": {
                "content": source_contract
            }
        },
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    allow_paths="."
)

# Trích xuất bytecode và ABI của MessageStorage
bytecode = compiled_sol["contracts"]["MessageStorage.sol"]["MessageStorage"]["evm"]["bytecode"]["object"]
abi = compiled_sol["contracts"]["MessageStorage.sol"]["MessageStorage"]["abi"]

# (Tùy chọn) Lưu JSON kết quả compile để tham khảo
with open("compiled_message_storage.json", "w", encoding="utf-8") as file_json:
    json.dump(compiled_sol, file_json, indent=4)

print("✅ Compile thành công. ABI và bytecode đã được trích xuất.")

# ================================
# 3. Kết nối tới Ganache
ganache_url = "http://127.0.0.1:7545"
w3 = Web3(Web3.HTTPProvider(ganache_url))
if not w3.is_connected():
    raise Exception("Không thể kết nối tới Ganache. Kiểm tra Ganache đã chạy chưa và port có đúng không?")

account_address = w3.eth.accounts[0]
chain_id = w3.eth.chain_id
nonce = w3.eth.get_transaction_count(account_address)

# 4. Triển khai contract
MessageStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

construct_txn = MessageStorage.constructor("Hello, Web3.py!").build_transaction({
    "from": account_address,
    "nonce": nonce,
    # --- CHỖ NÀY ĐÃ SỬA ---
    "gasPrice": w3.to_wei("20", "gwei"),
    "gas": 3000000,
    "chainId": chain_id
})

tx_hash = w3.eth.send_transaction(construct_txn)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Contract deployed at:", tx_receipt.contractAddress)

# 5. Tương tác với contract (ví dụ gọi setMessage)
deployed_contract = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

# Tăng nonce
nonce += 1
tx_set = deployed_contract.functions.setMessage("Một thông điệp mới!").build_transaction({
    "from": account_address,
    "nonce": nonce,
    # --- VÀO ĐÂY CŨNG PHẢI SỬA ---
    "gasPrice": w3.to_wei("20", "gwei"),
    "gas": 2000000,
    "chainId": chain_id
})

tx_set_hash = w3.eth.send_transaction(tx_set)
w3.eth.wait_for_transaction_receipt(tx_set_hash)
print("setMessage đã được mined.")
# 5. GỌI HÀM getMessage() ĐỂ ĐỌC THÔNG ĐIỆP BAN ĐẦU (VIEW)
# ================================
initial_message = deployed_contract.functions.getMessage().call()
print(f"🔍 Nội dung ban đầu trong contract: \"{initial_message}\"")

# ================================
# 6. GỌI HÀM setMessage() ĐỂ THAY ĐỔI THÔNG ĐIỆP (TRANSACTION)
# ================================
# Tăng nonce để gửi transaction kế tiếp
nonce += 1

# Build transaction gọi setMessage
tx_set_msg = deployed_contract.functions.setMessage("Một thông điệp mới từ Web3.py!").build_transaction({
    "from": account_address,
    "nonce": nonce,
    "gasPrice": w3.to_wei("20", "gwei"),
    "gas": 2000000,
    "chainId": chain_id
})

# Gửi transaction (Ganache GUI unlock sẵn nên không cần ký thủ công)
tx_set_hash = w3.eth.send_transaction(tx_set_msg)
print("⌛ Đang gửi transaction setMessage()...")

# Chờ transaction mined
tx_set_receipt = w3.eth.wait_for_transaction_receipt(tx_set_hash)
print("✅ setMessage() đã mined!")

# ================================
# 7. GỌI LẠI getMessage() ĐỂ XÁC NHẬN ĐÃ CẬP NHẬT
# ================================
updated_message = deployed_contract.functions.getMessage().call()
print(f"📝 Nội dung sau khi setMessage(): \"{updated_message}\"")


✅ Compile thành công. ABI và bytecode đã được trích xuất.
Contract deployed at: 0x638273B19c9B53d482Fd0a631C9A238faE707dc0
setMessage đã được mined.
🔍 Nội dung ban đầu trong contract: "Một thông điệp mới!"
⌛ Đang gửi transaction setMessage()...
✅ setMessage() đã mined!
📝 Nội dung sau khi setMessage(): "Một thông điệp mới từ Web3.py!"
